In [1]:
import os
%pwd

'g:\\End-to-End-Cancer-Cllassification-using-MLflow-and-DVC\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'g:\\End-to-End-Cancer-Cllassification-using-MLflow-and-DVC'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int 
    params_augmented: bool
    params_image_size: list

In [14]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        print(self.config)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->TrainingConfig:
        print(1)
        training=self.config.training
        print(training)
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"Chest-CT-Scan-data")
        create_directories([training.root_dir])
        print(training_data)

        model_trainer_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_augmented=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        print(1000)

        return model_trainer_config

In [21]:
import tensorflow as tf
class Training:
    def __init__(self,config=TrainingConfig):
        self.config=config

    def get_base_model(self):
        self.model=tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs=dict(
            rescale= 1./255,
            validation_split=0.20
       )
        
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_datagenerator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_augmented:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )

        else:
            train_datagenerator=valid_datagenerator

        self.train_datagenerator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='training',
            shuffle=True,
            **dataflow_kwargs
        )

    def train(self):
        self.step_per_epochs=self.train_datagenerator.samples//self.train_datagenerator.batch_size
        self.validation_steps=self.valid_datagenerator.samples//self.valid_datagenerator.batch_size

        early_Stopping=tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0.001,
            patience=3,
            verbose=1,
            mode='auto'
        )
        self.model.fit(self.train_datagenerator,
                       epochs=self.config.params_epochs,
                       steps_per_epoch=self.step_per_epochs,
                       validation_data=self.valid_datagenerator,
                       validation_steps=self.validation_steps,
                       callbacks=[early_Stopping]
                       )
        
        self.save_model(path=self.config.trained_model_path,model=self.model)

        
    @staticmethod
    def save_model(path:Path,model:tf.keras.models):
        model.save(str(path))

        

In [22]:
import sys
from cnnClassifier.exception import CustomException
try:
    config=ConfigurationManager()
    training_config=config.get_model_trainer_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    pass
except Exception as e:
    raise CustomException(e,sys)

[2024-07-30 07:24:21,330: INFO : common :yaml file:config\config.yaml loaded succesfully]
[2024-07-30 07:24:21,700: INFO : common :yaml file:params.yaml loaded succesfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/16Kt7T2Pm9BZf_1rd9a3AEAcQ_1BbmP_F/view?usp=drive_link', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prep_base_model', 'base_model_path': 'artifacts/prep_base_model/base_model.h5', 'updated_base_path': 'artifacts/prep_base_model/updated_base_model.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}}
[2024-07-30 07:24:21,762: INFO : common :Created directory at artifacts]
1
{'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}
[2024-07-30 07:24:21,765: INFO : common :Created directory at artifacts/